<a href="https://colab.research.google.com/github/aygul0790/Bootcamp/blob/main/Day_1/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers

### What we will cover today:

- Why should you care about Transformers?

- RNNs: Problems and progress

- Transformers: Context-aware embeddings

- Digging Deeper: What we missed

- The transformers Library

- Going further: GPT, BERT & other models

- Recap & Further reading

## 1️⃣ Why should you care about Transformers?

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/chatgpt.jpg?raw=1" width="500"/>

## 2️⃣ RNNs: Problems and progress

Before we dive into Transformers, let's remind ourselves how a lot of NLP tasks are often tackled by RNNs:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/rnn_problem.png?raw=1" width="750"/>

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/encoder_decoder.png?raw=1" width="750"/>


### What are the key issues we face here?

- Information bottleneck at interface
- Vanishing gradient problem
- We have to compute the entire sequence recursively (makes scaling very hard!)

### RNNs suffer from vanishing gradient through time

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/gradient_vanishing.png?raw=1" width="750"/>

❗️Backpropagation through time❗️ Within a single layer RNN model.

- During backpropagation, the gradient vanishes to 0 as the time step decreases.
- As a result, simple RNNs are said to have short memory (even with variants like LSTM and GRU)

📚 Michael Phi - [Illustrated Guide to Recurrent Neural Networks](https://medium.com/data-science/illustrated-guide-to-recurrent-neural-networks-79e5eb8049c9)

### What does this mean for our performance?

A simplification of problems with RNNs:

- Sally adored reading; when she received a book on her birthday she was older

What we'd like:

- Sally adored reading; when she received a book on her birthday she was happy!

RNNs are likely to miss out on **important context** from earlier in the sentence because of their recency bias 🫠

## 3️⃣ Transformers

### The paper that started it all: [Attention is All You Need](https://arxiv.org/abs/1706.03762)

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/attention_model.png?raw=1" width="400"/>

The highest level view:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/high_level.png?raw=1" width="500"/>

Broken down a bit more:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/encoder_decoder_simplified.png?raw=1" width="500"/>

### Before we talk about what's going on inside the encoder layers, let's talk about what's going into it!

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/numbered_embedding_diagram.png?raw=1" width="400"/>


Zooming in on one of the encoders:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/768_transformer_block.png?raw=1" width="400"/>


- Usually the **embedding size** is 768
- The **hidden dimension** (the length of the projected Q, K and V vectors) is also 768
- In the example we'll use size 2 for simplicity

❓ What's going on in this strange self-attention layer?

1) Each token (word) embedding gets **projected** ➡️ into 3 further vectors: the **query, key and value** vectors

2) We compute a **scaled dot-product** 🔴 on the query and key vectors to work out how much each word relates to those around it

3) Take these scores and **normalize with softmax** ⤵️

4) **Multiply by our value vectors** ❎, sum and pass to our dense neural network.

#### An example sentence

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/sentence.png?raw=1" width="600"/>

<br>

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/qkv_cleaned.png?raw=1" width="400"/>

Each of these three vectors are learned as the model sees more data!

#### Three zoomed-in examples:

Two words in a sentence that are closely related

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/dot_product_related_v2.png?raw=1" width="600"/>

The same two words but seen from the other perspective:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/dot_product_related_inverse_v2_1.png?raw=1" width="600"/>

Finally, two words with a weak connection:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/dot_product_unrelated_v2_1.png?raw=1" width="600"/>

#### Let's look at one dot product

To keep it really simple, we're going to imagine our Q, K and V have only been projected into two dimensions

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/01_one_Q_one_K_v2.png?raw=1" width="600"/>

#### What happens once we have our dot products?

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/02_one_Q_all_K_v2.png?raw=1" width="600"/>

#### Then we scale:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/03_one_Q_all_K_scaled_v2.png?raw=1" width="600"/>

#### Finally we apply softmax:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/04_one_Q_all_K_softmaxed_v2.png?raw=1" width="600"/>

<br>

We have to do this for each word in our sentence!

You can see how this becomes a matrix operation

We get the scaled dot-product attention for all of our Query and Key vectors:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/05_attention_matrix_v2.png?raw=1" width="600"/>

#### And then multiply our "similarity score" with all of our Value vectors

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/06_one_attention_all_V_v2_1.png?raw=1" width="600"/>

<br>

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/07_all_attention_all_V_v2_1.png?raw=1" width="600"/>

So really the entire thing can be written like this:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/QKV_matrix_method.png?raw=1" width="600"/>


We are done with our multiplications!

Now we just need to normalize and pass through to the feed-forward neural network

The neural network will output vectors of our original embedding dimension (e.g. 768)

### Putting it all together:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/slowest_qkv.gif?raw=1" width="950"/>

### Let's check the lecture notebook for some Tensorflow visualizations

Computing one set of all of these Q, K, V multiplications and processes is what we call "single-headed attention".

When we are doing our initial linear projections (used to create the Q, K and V vectors) we can express these operations as matrices of weights too!

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/QKV_matrix.png?raw=1" width="400"/>

With that in mind, we see the complexity of our model compared to the others:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/complexity.png?raw=1" width="400"/>

Context windows (a.k.a. our max sequence lengths) add a lot of weights, however they have been [getting much larger lately](https://www.anthropic.com/news/100k-context-windows)!

### Multi-headed?

- We can use multiple heads to split up and analyze different parts of our embedding
- Each can focus a different part of the embedding eg. working on semantic vs. syntactic features of our sentences

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/multi_headed.png?raw=1" width="400"/>

### Let's check in with our original diagram:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/one_encoder_block.png?raw=1" width="500"/>

#### What about the decoder?

At the highest level view, its job is to choose the **most likely next token**:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/decoder_high_veiw.png?raw=1" width="600"/>

### How does it do this? And what happened to all the work the encoder did?

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/decoder_training_step_1.png?raw=1" width="700"/>

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/decoder_training_step_2.png?raw=1" width="700"/>

### Cross-Attention

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/cross-attention-in-transformer-decoder.png?raw=1" width="700"/>


Attention between encoder and decoder (a.k.a. cross-attention):

- **Self-attention** operates within a single sequence and captures the relationships between tokens within that sequence.
- **Cross-attention** operates between two different sequences and captures the relationships between tokens from the source sequence and tokens from the target sequence, allowing the model to generate relevant output based on the information in the source sequence.

## Let's recap the training process at a high level

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/decoder_training_highest.png?raw=1" width="700"/>

### So what does inference look like?

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/inference.png?raw=1" width="700"/>

### That is all here!

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/attention_model.png?raw=1" width="400"/>

## Digging Deeper

#### What haven't we covered yet:
- Self-padding mask
- Skip layers
- Subword tokenization
- Positional encoding

#### Self-padding mask

- Masks out padding tokens during self-attention, preventing the model from attending to them and ensuring focus only on relevant parts of variable-length input sequences.

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/padding_in_practice_encoder.png?raw=1" width="700"/>

#### Skip Layers

- Information from earlier layers is propagated directly to later layers, aiding in the retention of valuable information during training.
- Faster convergence and improved performance ✅

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/skip_layers.png?raw=1" width="500"/>

### Subword tokenization
- Enables the model to handle OOV words and capture morphological variations (e.g. "walks" and "walking" aren't viewed as entirely separate tokens but rather composites of shared/ different tokens)
- Makes the language representation more compact, efficient, and generalizable across different word forms

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/subword.svg?raw=1" width="700"/>

### Positional encoding
- Gives the model an idea of each word's position in the sentence
- Can be hard-coded or learned
- Attention is All You Need has a clever hard-coding using varying frequencies to convey positional information

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/numbered_embedding_diagram.png?raw=1" width="400"/>

See [this video](https://www.youtube.com/watch?v=dichIcUZfOw) for a detailed walkthrough!

## 5️⃣ HuggingFace
Pretty tricky under the hood but the transformers makes it all super easy in code:

In [1]:
#!pip install transformers tensorflow

- Pre-trained Models: Provides a collection of pre-trained state-of-the-art models like BERT, GPT-2, T5, and many more for various NLP tasks.
- Simple: Offers a unified and simple way to find a model, fine-tune it, and deploy it.
- Multilingual: Supports models in multiple languages (although lots are primarily written in torch but there are plenty of tricks that enable you to work with PyTorch models)

In [ ]:
from transformers import pipeline
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

In [ ]:
result = pipe("I am a student and I am studying in London")
result[0]['translation_text']

### Under the hood of a pipeline
- Models use their own trained **tokenizers** which we load up with the `.from_pretrained()` method
- Then we just pass in the creator of the model and model name
- If we want to pass in Tensorflow tensors (which we will), just put TF before our model and pass `from_pt = True`.

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

tokens = tokenizer.encode("This is easy!", return_tensors = "tf")

print(tokens)

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr", from_pt = True)

output_tokens = model.generate(tokens)

print(output_tokens)

print(tokenizer.decode(output_tokens[0]))

## 6️⃣ Going further

### What are the two parts of our diagram doing?

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/decoder_training_highest.png?raw=1" width="700"/>

### The Transformer family tree:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/TF_fam_tree.png?raw=1" width="500"/>

- Encoder-only (e.g. BERT): converts an input sequence to a numerical representations. Uses words to the left and right of each word (hence "bidirectional") and is great for things like classification.

- Decoder-only (e.g. GPT): takes an input sequence and iteratively predicts the most likely next word (can also be used in a similar manner to encoder-decoder if trained correctly)

- Encoder-decoder (e.g. T5 or original "Attention is all you need" paper model): maps one sequence to another

### So what makes ChatGPT so great?

An example of the GPT-2 architecture:

<img src="https://github.com/aygul0790/Bootcamp/blob/main/Day_1/pics/GPT2.png?raw=1" width="500"/>

- Huge amounts of data and training
- 175B weights
- Reinforcement Learning from Human Feedback (RLHF): a technique used to train machine learning models, particularly in natural language processing (NLP), by incorporating human preferences into the training process. It's widely used in fine-tuning large language models (LLMs), such as ChatGPT, to align their responses with human expectations.

## 7️⃣Recap and Further Reading

- [Intro to Transformers](https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html#transformer_intro): A high level overview of many of the concepts covered today

- [Jay Alamar's Transformer Illustrated](https://jalammar.github.io/illustrated-transformer/): Fantastic step-by-step visualizations and explanations for multiple Transformer-based models

- [HuggingFace + Transformers Textbook](https://www.oreilly.com/library/view/natural-language-processing/9781098136789/): If you want to do anything with HuggingFace, this is a fantastic primer.

- [Accompanying Open Source GH Repo for the above textbook](https://github.com/nlp-with-transformers/notebooks): Totally free access to the examples from the above book!

- [StatsQuest Video on Self-Attention](https://www.youtube.com/watch?v=zxQyTK8quyY): A great step by step breakdown of the process